In [ ]:
pip uninstall tensorflow

Uninstalling tensorflow-2.2.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-2.2.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.2.0


In [ ]:
pip install tensorflow==1.13.2

     |████████████████████████████████| 92.6MB 51kB/s 
     |████████████████████████████████| 3.2MB 44.4MB/s 
     |████████████████████████████████| 368kB 46.4MB/s 
  Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0


In [ ]:
import tensorflow as tf
from tensorflow.contrib import keras
from tensorflow.contrib.keras import models
from tensorflow.contrib.keras import datasets
from tensorflow.contrib.keras import layers
from tensorflow.contrib.keras import preprocessing
from tensorflow.contrib.keras import backend as K
from tensorflow.contrib.keras import callbacks
from tensorflow.contrib.keras import utils

from collections import defaultdict
import pandas as pd
import numpy as np
import os
import json
import pickle
import sys
import random
import glob
from sklearn import metrics,model_selection
import csv
from collections import Counter
import warnings
import string
import re
import nltk
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

learn = tf.contrib.learn
tf.logging.set_verbosity(tf.logging.WARN)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [ ]:
MAX_DOCUMENT_LENGTH=100#128#100
maxlen_word=20#5#20
num_classes=4
embedding_dims=100
filters=250
kernel_size = 3
hidden_dims = 100
batch_size=128#5

In [ ]:
ann_dir= r"/content/train_2kmZucJ.csv" #set_combined_final

duplicate_samples=1
duplicate_samples_pos=1
#n_files=60
n_epoch=30
f= open("report_sections_cnn_only_text_09_01_17_files_"+"_epoch_"+str(n_epoch)+".txt","w")
#f.write("Number of files: ")
#f.write(str(n_files))
f.write("\nNumber of epoch: ")
f.write(str(n_epoch))
f.write("\n")
load_from_vector=False
model_dir="cnn_only_text_section_header/"

if not os.path.exists(model_dir):
    os.makedirs(model_dir)

In [ ]:
sh_dataset = defaultdict(lambda : None)
sh_dataset['target_names'] = ['pos','neg']
sh_dataset['target'] =[]
sh_dataset['data'] =[]
sh_dataset['layout'] =[]

In [ ]:
with open(ann_dir, 'rt',encoding="utf8") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        for i in range(duplicate_samples):
            if row['label'] =='0':
                for j in range(duplicate_samples_pos):
                    sh_dataset['target'].append(0)
                    sh_dataset['data'].append(row['tweet'])
            elif row['label'] =='1':
                for j in range(duplicate_samples_pos):
                    sh_dataset['target'].append(1)  
                    sh_dataset['data'].append(row['tweet'])


In [ ]:
row

OrderedDict([('id', '7920'),
             ('label', '0'),
             ('tweet',
              'Apple Barcelona!!! #Apple #Store #BCN #Barcelona #travel #iphone #selfie #fly #fun #cabincrew… http://instagram.com/p/wBApVzpCl3/')])

In [ ]:
# training data
training_dataset = defaultdict(lambda : None)
training_dataset['target_names'] =['pos','neg']
training_dataset['target'] =[]
training_dataset['data'] =[]

In [ ]:
len(sh_dataset['data'])

7920

In [ ]:
pos_sample = min(sh_dataset['target'].count(0),sh_dataset['target'].count(1))

count_top_sample=0
count_sub_sample=0
count_subsub_sample=0

In [ ]:
for i in range(len(sh_dataset['target'])):
    if sh_dataset['target'][i]==0:
        if count_top_sample<pos_sample:
            training_dataset['data'].append(sh_dataset['data'][i])
            training_dataset['target'].append(sh_dataset['target'][i])
            count_top_sample+=1
    elif sh_dataset['target'][i]==1:
        if count_sub_sample<pos_sample:
            training_dataset['data'].append(sh_dataset['data'][i])
            training_dataset['target'].append(sh_dataset['target'][i])
            count_sub_sample+=1

In [ ]:
training_dataset_ran = defaultdict(lambda : None)
training_dataset_ran['target_names'] =['pos','neg']
training_dataset_ran['target'] =[]
training_dataset_ran['data'] =[]

In [ ]:
import random
ran_index= random.sample(range(0, len(training_dataset['data'])), len(training_dataset['data']))
for i in ran_index:
    training_dataset_ran['data'].append(training_dataset['data'][i])
    training_dataset_ran['target'].append(training_dataset['target'][i])

In [ ]:
del training_dataset   
del sh_dataset

In [ ]:
print("Training")
f.write("Training\n")
print("Heading",training_dataset_ran["target"].count(0))
f.write("Top samples ")
f.write(str(training_dataset_ran["target"].count(1)))
f.write("\n")
print("Section Heading",training_dataset_ran["target"].count(1))
f.write("Top samples ")
f.write(str(training_dataset_ran["target"].count(1)))
f.write("\n")
print("Sub - Heading",training_dataset_ran["target"].count(2))
f.write("Sub samples ")
f.write(str(training_dataset_ran["target"].count(2)))
f.write("\n")
print("Sub-Sub Heading",training_dataset_ran["target"].count(3))
f.write("Subsub samples ")
f.write(str(training_dataset_ran["target"].count(3)))
f.write("\n")

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(sh_dataset['data'],sh_dataset['target'], test_size=0.2,random_state = 0)

In [ ]:
df_train=pd.DataFrame({'line':X_train,'class':y_train})
df_train.to_csv(model_dir+"stratified_training_50", sep='\t',index=False, encoding='utf-8')

df_test=pd.DataFrame({'line':X_test,'class':y_test})
df_test.to_csv(model_dir+"stratified_test_10", sep='\t',index=False, encoding='utf-8')

In [ ]:
import re,string

def strip_links(text):
    link_regex    = re.compile('((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)', re.DOTALL)
    links         = re.findall(link_regex, text)
    for link in links:
        text = text.replace(link[0], ', ')    
    return text
def convert_vulgar(text):
  words = []
  for word in text.split():
    if word=='$&@*#':
      words.append('fuck')
    else:
      words.append(word)
  return ' '.join(words)
def strip_all_entities(text):
    entity_prefixes = ['@']
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)
def remove_hastag(text):
  return text.replace('#', '')

In [ ]:
for i in range(len(X_train)):
  X_train[i] = strip_all_entities(strip_links(remove_hastag(convert_vulgar(X_train[i]))))
for i in range(len(X_test)):
  X_test[i] = strip_all_entities(strip_links(remove_hastag(convert_vulgar(X_test[i]))))

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
nltk.download('punkt')
stop_words = set(stopwords.words('english'))
def remove_stopwords(text):
  word_tokens = word_tokenize(text)
  Filtered_sentence = [w for w in word_tokens if not w in stop_words]
  return ' '.join(Filtered_sentence)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import spacy
nltk
nlp = spacy.load('en')
def removing_name_entity(text):
  document = nlp(text)
  text_no_namedentities = []
  ents = [e.text for e in document.ents]
  for item in document:
    if item.text in ents:
        pass
    else:
        text_no_namedentities.append(item.text)
  return " ".join(text_no_namedentities)

In [ ]:
for i in range(len(X_train)):
  X_train[i] = remove_stopwords(removing_name_entity(X_train[i]))
for i in range(len(X_test)):
  X_test[i] = remove_stopwords(removing_name_entity(X_test[i]))

In [ ]:
# Process vocabulary character level
char_processor = learn.preprocessing.ByteProcessor(MAX_DOCUMENT_LENGTH)
X_train_char = np.array(list(char_processor.fit_transform(X_train)))
X_test_char = np.array(list(char_processor.transform(X_test)))

In [ ]:
# Process vocabulary world level
vocab_processor = tf.contrib.learn.preprocessing.VocabularyProcessor(maxlen_word,min_frequency=100)
X_train_word = np.array(list(vocab_processor.fit_transform(X_train)))
X_test_word = np.array(list(vocab_processor.transform(X_test)))

In [ ]:
# #Class labels to categorical value
# y_train = keras.utils.to_categorical(y_train,num_classes=num_classes)
# y_test = keras.utils.to_categorical(y_test,num_classes=num_classes)

In [ ]:
max_words = len(vocab_processor.vocabulary_)
max_char = 256

In [ ]:
print(max_words)
print(max_char)

104
256


In [ ]:
maxlen_word

20

In [ ]:
# char_CNN
input_char =  layers.Input(shape=(MAX_DOCUMENT_LENGTH,), name = 'input_char')

embedding_char= layers.Embedding(max_char,embedding_dims,input_length=MAX_DOCUMENT_LENGTH)(input_char)
dropout1_char = layers.Dropout(0.2)(embedding_char)

conv1D_char = layers.Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1)(dropout1_char)

globalMaxPooling1_char= layers.GlobalMaxPooling1D()(conv1D_char)

dense_hidden_char = layers.Dense(hidden_dims)(globalMaxPooling1_char)
dropout2_char = layers.Dropout(0.2)(dense_hidden_char)
activation_char= layers.Activation('relu')(dropout2_char)

In [ ]:
main_output = layers.Dense(num_classes, activation='softmax', name='main_output')(activation_char)

In [ ]:
model = models.Model(inputs=[input_char], outputs=[main_output])

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_char (InputLayer)      (None, 100)               0         
_________________________________________________________________
embedding_7 (Embedding)      (None, 100, 100)          25600     
_________________________________________________________________
dropout_14 (Dropout)         (None, 100, 100)          0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 98, 250)           75250     
_________________________________________________________________
global_max_pooling1d_7 (Glob (None, 250)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 100)               25100     
_________________________________________________________________
dropout_15 (Dropout)         (None, 100)               0         
__________

In [ ]:
filepath=model_dir+"weights-improvement-{epoch:02d}-{val_acc:.9f}.hdf5"
checkpoint = callbacks.ModelCheckpoint(filepath, monitor='F1_score', verbose=1, save_best_only=True, mode='max')
history = callbacks.History()

In [ ]:
model.fit(X_train_char, y_train,
          batch_size=batch_size,shuffle=True,
          epochs=13,verbose=1,
          validation_data=(X_test_char, y_test),callbacks=[checkpoint,history])

Train on 6336 samples, validate on 1584 samples
Epoch 1/13
6336/6336 [==============================] - 13s 2ms/sample - loss: 0.6479 - acc: 0.7424 - val_loss: 0.4071 - val_acc: 0.8201
Epoch 2/13
6336/6336 [==============================] - 13s 2ms/sample - loss: 0.3905 - acc: 0.8224 - val_loss: 0.3517 - val_acc: 0.8485
Epoch 3/13
6336/6336 [==============================] - 13s 2ms/sample - loss: 0.3334 - acc: 0.8580 - val_loss: 0.3141 - val_acc: 0.8706
Epoch 4/13
6336/6336 [==============================] - 13s 2ms/sample - loss: 0.3026 - acc: 0.8736 - val_loss: 0.3031 - val_acc: 0.8681
Epoch 5/13
6336/6336 [==============================] - 13s 2ms/sample - loss: 0.2869 - acc: 0.8780 - val_loss: 0.2972 - val_acc: 0.8756
Epoch 6/13
6336/6336 [==============================] - 13s 2ms/sample - loss: 0.2784 - acc: 0.8824 - val_loss: 0.2914 - val_acc: 0.8763
Epoch 7/13
6336/6336 [==============================] - 13s 2ms/sample - loss: 0.2616 - acc: 0.8913 - val_loss: 0.2938 - val_acc: 

In [ ]:
yhat_probs = model.predict(X_test_char, verbose=0)

In [ ]:
y_pred = []
for i in np.arange(len(yhat_probs)):
    y_pred.append(yhat_probs[i][0])

In [ ]:
test = pd.read_csv('test_oJQbWVk.csv')

In [ ]:
x_test = test['tweet']

In [ ]:
for i in range(len(X_test)):
  x_test[i] = strip_all_entities(strip_links(remove_hastag(convert_vulgar(x_test[i]))))

In [ ]:
for i in range(len(X_test)):
  x_test[i] = remove_stopwords(removing_name_entity(x_test[i]))

In [ ]:
x_test_char = np.array(list(char_processor.transform(x_test)))

In [ ]:
x_test_words = np.array(list(vocab_processor.transform(x_test)))

In [ ]:
#confusion matrix
from sklearn.metrics import classification_report, confusion_matrix
yhat_probs1 = model.predict(x_test_char, verbose=0)

In [ ]:
y_pred1 = []
for i in np.arange(len(yhat_probs1)):
    y_pred1.append(yhat_probs1[i][0])

In [ ]:
C = pd.DataFrame()
C1 = pd.DataFrame()
C['f1'] = y_pred
C1['f1'] = y_pred1

In [ ]:
C['f2'] = y_pred
C1['f2'] = y_pred1

In [ ]:
C['f3'] = y_pred
C1['f3'] = y_pred1

In [ ]:
C['f4'] = y_pred
C1['f4'] = y_pred1

In [ ]:
C['f5'] = y_pred
C1['f5'] = y_pred1

In [ ]:
C['f6'] = y_pred
C1['f6'] = y_pred1

In [ ]:
C['f7'] = y_pred
C1['f7'] = y_pred1

In [ ]:
C['f8'] = y_pred
C1['f8'] = y_pred1

In [ ]:
C['r'] = y_test

In [ ]:
C

,f1,f2,f3,f4,f5,f6,f7,f8,r
0,0.998416,0.607817,0.998116,0.999202,0.998680,0.971961,0.996768,0.999511,0
1,0.057165,0.157360,0.231886,0.052355,0.059222,0.175998,0.152385,0.042954,1
2,0.948777,0.748943,0.987241,0.964680,0.945628,0.680139,0.815134,0.309549,0
3,0.418235,0.435295,0.116457,0.137114,0.524201,0.104070,0.248611,0.437710,0
4,0.988883,0.812548,0.993853,0.976787,0.982364,0.869260,0.971748,0.965718,0
...,...,...,...,...,...,...,...,...,...
1579,0.179476,0.718851,0.427883,0.073128,0.102843,0.773416,0.512399,0.115211,0
1580,0.546571,0.907670,0.883691,0.735058,0.537632,0.843690,0.624340,0.712817,0
1581,0.999154,0.889502,0.990393,0.999466,0.999819,0.848743,0.962125,0.997168,0
1582,0.999822,0.950064,0.999170,0.999576,0.999893,0.991842,0.998986,0.998892,0


In [ ]:
C.to_csv('125.csv')
C1.to_csv('124.csv')

In [ ]:
import pandas as pd

In [ ]:
C = pd.read_csv('125.csv')
C1 = pd.read_csv('124.csv')

In [ ]:
C = C.iloc[:,1:]
C1 = C1.iloc[:,1:]


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_tr, x_te, y_tr, y_te = train_test_split(C.iloc[:,:-1],C.iloc[:,8], test_size=0.25,random_state = 10)

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(x_tr, y_tr)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
y = model.predict_proba(x_te)

In [ ]:
y1 = model.predict_proba(C1)

In [ ]:
y_pred = []
for i in range(len(y)):
    y_pred.append(y[i][0])

In [ ]:
y_pred1 = []
for i in range(len(y1)):
    y_pred1.append(y1[i][0])

In [ ]:
A = pd.DataFrame()
A1 = pd.DataFrame()
A['f1'] = y_pred
A1['f1'] = y_pred1

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model= RandomForestClassifier(n_estimators=1000)
model.fit(x_tr, y_tr)
y_p = model.predict_proba(x_te)

In [ ]:
y1 = model.predict_proba(C1)

In [ ]:
y_pred = []
for i in range(len(y_p)):
    y_pred.append(y_p[i][0])

In [ ]:
y_pred1 = []
for i in range(len(y1)):
    y_pred1.append(y1[i][0])

In [ ]:
A['f2'] = y_pred
A1['f2'] = y_pred1

In [ ]:
A['f3'] = y_pred
A1['f3'] = y_pred1

In [ ]:
y_pred = classifier.predict(C1)
for i in range(0,len(y_pred)):    
  if y_pred[i]>=.5:         
     y_pred[i]=1    
  else:         
    y_pred[i]=0

In [ ]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB(var_smoothing=1e-08)
classifier.fit(x_tr, y_tr)

# Predicting the Test set results
y_pred = classifier.predict_proba(x_te)

In [ ]:
y_pred1 = classifier.predict_proba(C1)

In [ ]:
y = []
for i in range(0,len(y_pred)):            
     y.append(y_pred[i][0])

In [ ]:
y1 = []
for i in range(0,len(y_pred1)):            
     y1.append(y_pred1[i][0])

In [ ]:
A['f4'] = y
A1['f4'] = y1

In [ ]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'linear', random_state = 0)
classifier.fit(x_tr, y_tr)

# Predicting the Test set results
y_pred = classifier.predict(x_te)

In [ ]:
y_pred1 = classifier.predict(C1)

In [ ]:
A['f5'] = 1-y_pred
A1['f5'] = 1-y_pred1

In [ ]:
import lightgbm as lgb 
param = {'num_leaves':150, 'objective':'binary','max_depth':7,'learning_rate':.06,'max_bin':200} 
param['metric'] = ['auc', 'binary_logloss']
num_round=50 
train_data=lgb.Dataset(x_tr,label=y_tr)
lgbm=lgb.train(param,train_data,num_round) 
ypred2=lgbm.predict(x_te)

In [ ]:
A['f6'] = 1-ypred2

In [ ]:
y_pred = lgbm.predict(C1)

In [ ]:
A1['f6'] = 1-y_pred

In [ ]:
for i in range(0,len(y_pred)):    
  if y_pred[i]>=.5:         
     y_pred[i]=1    
  else:         
    y_pred[i]=0

In [ ]:
x_tr, x_te, y_tr, y_te = train_test_split(A,y_te, test_size=0.25,random_state = 10)

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(x_tr, y_tr)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
y = model.predict_proba(x_te)

In [ ]:
z = []
for i in range(0,len(y)):    
  if y[i][0]>=.4775:         
     z.append(0)    
  else:         
    z.append(1)

In [ ]:
y = model.predict_proba(A1)

In [ ]:
sub = pd.DataFrame()
sub['id'] = test['id']
sub['label'] = z

In [ ]:
sub.to_csv('sub31.csv', index=False)

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report
cm = confusion_matrix(y_te, z)
cm

array([[63,  9],
       [ 4, 23]])

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model= RandomForestClassifier(n_estimators=1000)
model.fit(x_tr, y_tr)
y_p = model.predict(x_te)

In [ ]:
print(classification_report(y_te, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.87      0.92       285
           1       0.73      0.92      0.82       111

    accuracy                           0.88       396
   macro avg       0.85      0.89      0.87       396
weighted avg       0.90      0.88      0.89       396



In [ ]:
model.load_weights('./cnn_only_text_section_header/weights-improvement-20-0.824622512.hdf5')

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.97      0.98      0.98       304
           1       0.84      0.64      0.73       120
           2       0.65      0.46      0.54       144
           3       0.74      0.91      0.82       293

    accuracy                           0.82       861
   macro avg       0.80      0.75      0.77       861
weighted avg       0.82      0.82      0.82       861

